In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

References:
1. https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [2]:
# Settings:
pd.set_option('display.width', 190)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 200)
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('default')
np.set_printoptions(threshold = 30, edgeitems = 30, precision = 2, suppress = False)


In [3]:
def split_sequences(Xs, ys, n_steps):
    X, y = list(), list()
    for i in range(len(ys)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(ys):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = Xs[i: end_ix], ys[end_ix - 1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y).squeeze()

In [4]:
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from skorch import NeuralNetBinaryClassifier


class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()

        self.lstm = nn.LSTM(input_size=15, hidden_size=6, num_layers=1, batch_first=True)
        self.dense = nn.Linear(6, 1)

    def forward(self, X, **kwargs):
        output, hidden = self.lstm(X)
        X = self.dense(output[:, -1, :])
        return X

def get_model():
    model = NeuralNetBinaryClassifier(
        MyModule,
        optimizer=Adam,
        max_epochs=50,
        lr=3e-4,
        batch_size=16,
        iterator_train__shuffle=True,
    )
    return model

In [5]:
df_path = "../merged_data/features_USRECD.csv"
features = ["BCI", "BCIp", "BCIg", 'IE_SP_Comp', 'IE_SP_Dividend', 'IE_SP_Earnings', 'IE_Consumer_CPI', 'IE_Long_Interest', 'IE_Real_Price', 'IE_Real_Dividend', 'IE_Return_Price', 'IE_Real_Earnings',
                'IE_Scaled_Earnings', 'IE_Monthly_Returns', 'IE_Real_Returns']


In [6]:
# Read the data and do a little bit of wrangling:
df = pd.read_csv(df_path)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date", drop=True)
df = df.drop(columns="Unnamed: 0")
df.head()

,BCI,BCIp,BCIg,USRECD,IE_SP_Comp,IE_SP_Dividend,IE_SP_Earnings,IE_Consumer_CPI,IE_Long_Interest,IE_Real_Price,IE_Real_Dividend,IE_Return_Price,IE_Real_Earnings,IE_Scaled_Earnings,IE_Monthly_Returns,IE_Real_Returns
Date,,,,,,,,,,,,,,,,
1967-02-09,4.6052,6.5870,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
1967-02-16,4.6052,6.5863,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
1967-02-23,4.6012,6.5774,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
1967-03-02,4.6032,6.5820,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153
1967-03-09,4.6042,6.5852,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153


In [7]:
# Split into training and test sets and hold out the test set until the end, so that it remains "unseen".
lag_of_y = 21 # This is the lag we introduce to the target variable so that we assess the indicator's 
              # ability to predict the target variable this many steps into the future.
              # With BCI, a lag of 21 data points corresponds to about half a year.
        
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:-lag_of_y, df.columns != "USRECD"], \
    df.iloc[lag_of_y:, df.columns == "USRECD"], test_size=0.1, shuffle=False)

In [8]:
# Do a time series cross-validation on the test set by splitting it to k folds and doing a "rolling"
# validation against a validation fold, then averaging out the metrics.
splits = 4 # This is the number of splits/folds in the rolling validation.
tscv = TimeSeriesSplit(n_splits=splits)

for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29 ... 471 472 473 474 475
 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493
 494 495 496 497 498 499 500] TEST: [ 501  502  503  504  505  506  507  508  509  510  511  512  513  514
  515  516  517  518  519  520  521  522  523  524  525  526  527  528
  529  530 ...  971  972  973  974  975  976  977  978  979  980  981  982
  983  984  985  986  987  988  989  990  991  992  993  994  995  996
  997  998  999 1000]
TRAIN: [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29 ...  971  972  973  974  975  976  977  978  979  980  981  982
  983  984  985  986  987  988  989  990  991  992  993  994  995  996
  997  998  999 1000] TEST: [1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014
 1015 1016 1017 1018 1019 1020 102

## Validation

In [9]:
AUC_ROCs = dict()
ACCs = dict()
model_name = "LSTM"
print(model_name)
AUC_ROCs[model_name] = 0
ACCs[model_name] = 0
for train_index, test_index in tscv.split(X_train): # Rolling cross-validation happens inside this loop.
    X_train_fold, X_validation_fold = X_train.iloc[train_index[:-lag_of_y], X_train.columns != "USRECD"], \
        X_train.iloc[test_index[:-lag_of_y], X_train.columns != "USRECD"]
    y_train_fold, y_validation_fold = y_train.iloc[train_index[lag_of_y:], y_train.columns == "USRECD"], \
        y_train.iloc[test_index[lag_of_y:], y_train.columns == "USRECD"]

    scalers = dict()
    for feature in features:
        scalers[feature] = StandardScaler()
        scalers[feature].fit(X_train_fold[[feature]])
        X_train_fold[feature] = scalers[feature].transform(X_train_fold[[feature]])
        X_validation_fold[feature] = scalers[feature].transform(X_validation_fold[[feature]])

    X_train_fold, y_train_fold = split_sequences(X_train_fold.to_numpy(), y_train_fold.to_numpy(), n_steps=10)
    X_train_fold = X_train_fold.astype(np.float32)
    y_train_fold = y_train_fold.astype(np.float32)
    X_validation_fold, y_validation_fold = split_sequences(X_validation_fold.to_numpy(), y_validation_fold.to_numpy(), n_steps=10)
    X_validation_fold = X_validation_fold.astype(np.float32)
    y_validation_fold = y_validation_fold.astype(np.float32)
    model = get_model()
    model.fit(X_train_fold, y_train_fold)
    positive_probs = [p[1] for p in model.predict_proba(X_validation_fold)]
    AUC_ROC = metrics.roc_auc_score(y_validation_fold, positive_probs)
    AUC_ROCs[model_name] += AUC_ROC
    predictions = model.predict(X_validation_fold)
    ACC = accuracy_score(y_validation_fold, predictions)
    ACCs[model_name] += ACC
    print(AUC_ROC, ACC)

AUC_ROCs[model_name] /= splits
ACCs[model_name] /= splits

LSTM
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6374       0.7474        0.6683  0.0251
      2        0.6248       0.7474        0.6651  0.0249
      3        0.6132       0.7474        0.6606  0.0243
      4        0.6019       0.7474        0.6558  0.0239
      5        0.5912       0.7474        0.6510  0.0236
      6        0.5803       0.7474        0.6471  0.0236
      7        0.5693       0.7474        0.6437  0.0241
      8        0.5573       0.7474        0.6405  0.0236
      9        0.5447       0.7474        0.6378  0.0242
     10        0.5317       0.7474        0.6349  0.0235
     11        0.5177       0.7474        0.6318  0.0241
     12        0.5032       0.7474        0.6291  0.0236
     13        0.4883       0.7474        0.6254  0.0238
     14        0.4731       0.7474        0.6214  0.0234
     15        0.4582       0.7474        0.6174  0.0235
     16        0.4437     

In [10]:
print(model_name)
print(f"AUC ROC: {AUC_ROCs[model_name]}")
print(f"accuracy: {ACCs[model_name]}")

LSTM
AUC ROC: 0.6720388934395481
accuracy: 0.6457446808510638


## Test

In [11]:
# random guess
total = y_train.shape[0]
metrics.roc_auc_score(y_train.USRECD, np.zeros(total)), accuracy_score(y_train.USRECD, np.zeros(total))

(0.5, 0.8560575769692124)

In [12]:
X_train = X_train.copy()
X_test = X_test.copy()

all_scalers = dict()
for feature in features:
    all_scalers[feature] = StandardScaler()
    all_scalers[feature].fit(X_train[[feature]])
    X_train[feature] = all_scalers[feature].transform(X_train[[feature]])
    X_test[feature] = all_scalers[feature].transform(X_test[[feature]])

In [13]:
for feature in features:
    print(all_scalers[feature].mean_)

[4.8]
[6.44]
[3.61]
[5.88]
[2.29]
[3.06]
[4.71]
[1.81]
[6.78]
[3.18]
[12.51]
[3.96]
[9.69]
[0.01]
[2.96]


In [14]:
print(model_name)
model = get_model()

X_train, y_train = split_sequences(X_train.to_numpy(), y_train.to_numpy(), n_steps=10)
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

X_test, y_test = split_sequences(X_test.to_numpy(), y_test.to_numpy(), n_steps=10)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)
    
model.fit(X_train, y_train)
positive_probs = [p[1] for p in model.predict_proba(X_test)]
AUC_ROC = metrics.roc_auc_score(y_test, positive_probs)
print(AUC_ROC)
predictions = model.predict(X_test)
ACC = accuracy_score(y_test, predictions)
print(ACC)

LSTM
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7726       0.1603        0.7683  0.1216
      2        0.6450       0.3567        0.7467  0.1188
      3        0.5354       0.6192        0.7164  0.1185
      4        0.4259       0.7335        0.6444  0.1202
      5        0.3415       0.7275        0.5340  0.1210
      6        0.2870       0.7234        0.4826  0.1211
      7        0.2585       0.7275        0.4456  0.1192
      8        0.2384       0.7315        0.4319  0.1186
      9        0.2234       0.7275        0.4155  0.1191
     10        0.2107       0.7214        0.4125  0.1183
     11        0.1986       0.7174        0.4119  0.1184
     12        0.1897       0.7154        0.4121  0.1185
     13        0.1824       0.7134        0.4121  0.1179
     14        0.1755       0.7335        0.4082  0.1183
     15        0.1683       0.7315        0.4096  0.1183
     16        0.1622     